<a href="https://colab.research.google.com/github/nichakornchaisuwan/Project_Boneage/blob/main/train_Bonet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Standard lib imports
import os
import csv
import glob
import time
import argparse
import warnings
import pandas as pd
import os.path as osp

In [3]:
!pip install horovod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.5 MB 16.9 MB/s 
  Created wheel for horovod: filename=horovod-0.26.1-cp38-cp38-linux_x86_64.whl size=28369948 sha256=8fc677c1c6cb12fca99d91b3f4e49fc79a45ac53008902d12b7a47da73cca5d7
  Stored in directory: /root/.cache/pip/wheels/eb/0b/90/d53058f75f3ae3db9557f3e55dd8c016b2397e9b38557c8b66
Successfully built horovod


In [4]:
! git clone https://github.com/BCV-Uniandes/Bonet.git

Cloning into 'Bonet'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (52/52), done.


In [9]:
cd Bonet


/content/Bonet


In [26]:
!pip install conda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 299 kB 19.5 MB/s 
     |████████████████████████████████| 66 kB 5.4 MB/s 
     |████████████████████████████████| 109 kB 51.5 MB/s 
     |████████████████████████████████| 555 kB 43.5 MB/s 
  Created wheel for conda: filename=conda-4.3.16-py3-none-any.whl size=336959 sha256=aaa0b3ab337b8af7b569fc35f56305e8987b63542a3263773f2b63ef251a79d0
  Stored in directory: /root/.cache/pip/wheels/6e/c5/70/e50a1a99d26cbe940549549b7601d484e0f0bc4b8f0ec49e36
  Created wheel for pycosat: filename=pycosat-0.6.3-cp38-cp38-linux_x86_64.whl size=143278 sha256=788806493552098949fcead6d939015ce5587a551b3408fa72456c60386175f5
  Stored in directory: /root/.cache/pip/wheels/71/5b/2d/0a9247760f0f008abc0eae7c0127a8c1cddfe1145e4b170fc0
Successfully built conda pycosat


In [31]:
nvidia-smi

NameError: ignored

In [22]:
python test.py --data-test $DATA_TEST --ann-path-test $ANN_PATH_TEST --rois-path-test $ROIS_PATH_TEST --batch-size $BATCH_SIZE --gpu $GPUS --save-folder $SAVE_FOLDER --snapshot $SNAPSHOT --dataset $DATASET --heatmap --workers $NUM_WORKERS --cropped

SyntaxError: ignored

In [ ]:
# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
import horovod.torch as hvd
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

In [ ]:
# Other imports
from tqdm import tqdm
import pdb

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
parser = argparse.ArgumentParser()

In [ ]:
# Dataloading-related settings
parser.add_argument('--heatmaps', default=False, action='store_true',
                help='Train model with gaussian heatmaps')
parser.add_argument('--cropped', default=False, action='store_true',
                help='Train model with cropped images according to bbox')
parser.add_argument('--dataset', default='RSNA', type=str,choices=['RSNA','RHPE'],
                help='Dataset to perform training')

_StoreAction(option_strings=['--dataset'], dest='dataset', nargs=None, const=None, default='RSNA', type=<class 'str'>, choices=['RSNA', 'RHPE'], help='Dataset to perform training', metavar=None)

In [ ]:
parser.add_argument('--data-train', default='data/train/', type=str,
                help='path to train data folder')
parser.add_argument('--ann-path-train', default='train.csv', type=str,
                help='path to BAA annotations file')
parser.add_argument('--rois-path-train', default='train.json',
                type=str, help='path to ROIs annotations in coco format')

_StoreAction(option_strings=['--rois-path-train'], dest='rois_path_train', nargs=None, const=None, default='train.json', type=<class 'str'>, choices=None, help='path to ROIs annotations in coco format', metavar=None)

In [ ]:
parser.add_argument('--data-val', default='data/val/', type=str,
                help='path to val data folder')
parser.add_argument('--ann-path-val', default='val.csv', type=str,
                help='path to BAA annotations file')
parser.add_argument('--rois-path-val', default='val.json',
                type=str, help='path to ROIs annotations in coco format')

_StoreAction(option_strings=['--rois-path-val'], dest='rois_path_val', nargs=None, const=None, default='val.json', type=<class 'str'>, choices=None, help='path to ROIs annotations in coco format', metavar=None)

In [ ]:
parser.add_argument('--save-folder', default='TRAIN/new_test/',
                help='location to save checkpoint models')
parser.add_argument('--snapshot', default='boneage_bonet_weights.pth',
                help='path to weight snapshot file')
parser.add_argument('--optim-snapshot', type=str,
                default='boneage_bonet_optim.pth',
                help='path to optimizer state snapshot')

_StoreAction(option_strings=['--optim-snapshot'], dest='optim_snapshot', nargs=None, const=None, default='boneage_bonet_optim.pth', type=<class 'str'>, choices=None, help='path to optimizer state snapshot', metavar=None)

In [ ]:
parser.add_argument('--eval-first', default=False, action='store_true',
                help='evaluate model weights before training')
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                help='number of data loading workers (default: 4)')

_StoreAction(option_strings=['-j', '--workers'], dest='workers', nargs=None, const=None, default=4, type=<class 'int'>, choices=None, help='number of data loading workers (default: 4)', metavar='N')

In [ ]:
# Training procedure settings
parser.add_argument('--batch-size', default=1, type=int,
                help='Batch size for training')
parser.add_argument('--epochs', type=int, default=20,
                help='upper epoch limit')
parser.add_argument('--lr', '--learning-rate', default=1e-5, type=float,
                help='initial learning rate')
parser.add_argument('--patience', default=2, type=int,
                help='patience epochs for LR decreasing')
parser.add_argument('--start-epoch', type=int, default=1,
                help='epoch number to resume')
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed')
parser.add_argument('--log-interval', type=int, default=30, metavar='N',
                    help='report interval')

_StoreAction(option_strings=['--log-interval'], dest='log_interval', nargs=None, const=None, default=30, type=<class 'int'>, choices=None, help='report interval', metavar='N')

In [ ]:
parser.add_argument('--gpu', type=str, default='2,3')

_StoreAction(option_strings=['--gpu'], dest='gpu', nargs=None, const=None, default='2,3', type=<class 'str'>, choices=None, help=None, metavar=None)

In [ ]:
args,unknown = parser.parse_known_args()

In [ ]:
args_dict = vars(args)
print('Argument list to program')
print('\n'.join(['--{0} {1}'.format(arg, args_dict[arg])
                 for arg in args_dict]))
print('\n\n')

torch.manual_seed(args.seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu

if not os.path.exists(args.save_folder):
    os.makedirs(args.save_folder)

Argument list to program
--heatmaps False
--cropped False
--dataset RSNA
--data_train data/train/
--ann_path_train train.csv
--rois_path_train train.json
--data_val data/val/
--ann_path_val val.csv
--rois_path_val val.json
--save_folder TRAIN/new_test/
--snapshot boneage_bonet_weights.pth
--optim_snapshot boneage_bonet_optim.pth
--eval_first False
--workers 4
--batch_size 1
--epochs 20
--lr 1e-05
--patience 2
--start_epoch 1
--seed 1111
--log_interval 30
--gpu 2,3





In [ ]:
# Horovod settings
hvd.init()
torch.cuda.set_device(hvd.local_rank())
torch.cuda.manual_seed(hvd.size())

args.distributed = hvd.size() > 1
args.rank = hvd.rank()
args.size = hvd.size()

RuntimeError: ignored